In [1]:
from os import environ

import pandas as pd
from dotenv import load_dotenv
from langchain import OpenAI

load_dotenv();

In [2]:
from langchain import OpenAI

OPENAI_KEY = environ['OPENAI_KEY']

llm = OpenAI(temperature=0, openai_api_key=OPENAI_KEY)

### Import and clean reviews

In [4]:
df_reviews = pd.read_csv('./data/g2.csv')

In [5]:


def extract_answers(df_original):
    df = df_original.copy()
    # Define the regular expressions for each question
    regex_like_best = r"What do you like best about .+?\?\n\n(.*?)(?:\n\n|$)"
    regex_dislike = r"What do you dislike about .+?\?\n\n(.*?)(?:\n\n|$)"
    regex_problems_solving = r"What problems is .+? solving and how is that benefiting you\?\n\n(.*?)(?:\n\n|$)"

    # Use str.extract method to extract the answers and create new columns
    df['like_best'] = df['Review body'].str.extract(regex_like_best, expand=False)
    df['dislike'] = df['Review body'].str.extract(regex_dislike, expand=False)
    df['problems_solving'] = df['Review body'].str.extract(regex_problems_solving, expand=False)

    return df

df_augmented = extract_answers(df_reviews)
df_augmented

,Name,Review slug,Company name,Competitor type,Review rating,Review Link,Reviewer Type,Reviewer Title,Review title,Review body (before company name normalization),...,Verified Current User?,Incentivized Review,Review Source,Published date,Original date,Published date == Original date,ID,like_best,dislike,problems_solving
0,Hemant M.,anaplan,Anaplan,Enterprise Platform,10,https://www.g2.com/survey_responses/anaplan-re...,Small-Business(50 or fewer emp.),Director,Anaplan really is a game changer for finance a...,What do you like best about Anaplan?\n\nAnapla...,...,1,0,Organic,09-Aug-23,09-Aug-23,1,8432500,Anaplan is a fantastic tool to model complex b...,"Anaplan is wonderful in many many ways, but as...",The core problem businesses face is in spreads...
1,Paul P.,anaplan,Anaplan,Enterprise Platform,10,https://www.g2.com/survey_responses/anaplan-re...,Mid-Market(51-1000 emp.),Director,"Anaplan is Flexible and Powerful, But Does Not...",What do you like best about Anaplan?\n\nExcel ...,...,1,0,Organic,18-Jul-23,18-Jul-23,1,8334645,Excel is great when you are working by yoursel...,The initial price point is pretty high. It is ...,Anaplan helps our clients connect their Financ...
2,Amit C.,anaplan,Anaplan,Enterprise Platform,10,https://www.g2.com/survey_responses/anaplan-re...,Mid-Market(51-1000 emp.),NaN,Using Anaplan for connected Supply Chain Plann...,What do you like best about Anaplan?\n\nAnapla...,...,1,0,Organic,22-May-23,03-Jun-20,0,4249990,Anaplan is relatively low cost platform which ...,"Nothing to dislike, I think Anaplan is a great...",Problems solved - Rule based allocation / Allo...
3,Verified User in ConsultingInformation,anaplan,Anaplan,Enterprise Platform,8,https://www.g2.com/survey_responses/anaplan-re...,Small-Business(50 or fewer emp.),NaN,Anaplan : A great connected planning platform,What do you like best about Anaplan?\n\nAnapla...,...,1,0,Organic,06-Aug-23,06-Aug-23,1,8410489,Anaplan is a lightweight solution which does n...,There is no ETL tools embedded (excpet for pay...,It helps our customers to solve the lack of mu...
4,Verified User in Computer SoftwareInformation,anaplan,Anaplan,Enterprise Platform,9,https://www.g2.com/survey_responses/anaplan-re...,Small-Business(50 or fewer emp.),NaN,Anaplan is a fantastic planning solution for b...,What do you like best about Anaplan?\n\nIt is ...,...,1,0,Organic,01-Aug-23,26-Oct-20,0,4439386,It is simple to build (point and click instead...,Anaplan has a tendency to talk the talk about ...,"Sales forecasting, P&L forecasting, new item d..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2306,Farhan H.,onplan-platform,OnPlan,Next Gen Platform,10,https://www.g2.com/survey_responses/onplan-rev...,Small-Business(50 or fewer emp.),Veterinarian,Best ever software for life,What do you like best about OnPlan?\n\nBusines...,...,0,0,Organic,13-Apr-23,13-Apr-23,1,7942048,Business plan created by on plan seems like it...,Rarely it takes time but it's not a very big d...,I am tried to calculate my budget for purchasi...
2307,Brian D.,onplan-platform,OnPlan,Next Gen Platform,10,https://www.g2.com/survey_responses/onplan-rev...,Small-Business(50 or fewer emp.),NaN,"After much searching, I found the solution.",What do you like best about OnPlan?\n\nOnPlan ...,...,1,0,Organic,17-Apr-23,17-Apr-23,1,7956317,OnPlan uses the client base model to bring hig...,OnPlan requires the active participation of th...,We built project level cost accounting from di...
2308,Michelle A.,onplan-platform,OnPlan,Next Gen Platform,10,https://www.g2.com/survey_responses/onplan-rev...,Mid-Market(51-1000 emp.),Operations Manager,An extremely potent platform,What do you like best about OnPlan?\n\nOnPlan ...,...,0,0,Organic,19-Apr-23,19-Apr-23,1,7970332,"OnPlan is a dynamic, effective application tha...","Honestly, nothing. This is an excellent platfo...",Certain accounting-related tasks are greatly m...
2309,Allan Andrei S.,onplan-platform,OnPlan,Next Gen Platform,

In [6]:
!pip install spacy
!python -m spacy download en_core_web_sm 


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
# Let's split the reviews into sentences so we can get a fine-grained view of the reviews
import spacy

# Load the small English model. You can use the medium or large model for more accuracy but at the cost of speed.
nlp = spacy.load("en_core_web_sm")

def split_into_sentences(text):
    # Use the model to split the text into sentences
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

In [8]:
# Create a copy of the ID column to preserve the original row ordering
sm_df = df_augmented[df_augmented['Company name'] == 'Anaplan']
df = sm_df.copy()

# Create separate DataFrames for each review type
base_df = df_augmented[['ID', 'Name', 'Review slug', 'Company name', 'Competitor type',
       'Review rating', 'Review Link', 'Reviewer Type', 'Reviewer Title',
       'Review title', 
       'Business Partner Review?', 'Validated Reviewer?',
       'Verified Current User?', 'Incentivized Review', 'Review Source',
       'Published date', 'Original date', 'Published date == Original date',
       ]]
like_best_df = df[['ID', 'like_best']].copy()
dislike_df = df[['ID', 'dislike']].copy()
problems_solving_df = df[['ID', 'problems_solving']].copy()

# Define a function to process each DataFrame
def explode_reviews(df, column_name):
    # Split reviews into sentences
    df[column_name] = df[column_name].astype(str).apply(split_into_sentences)
    
    # Explode the DataFrame and reset the index
    df = df.explode(column_name).reset_index(drop=True)
    
    return df

# # Process each DataFrame
like_best_df = explode_reviews(like_best_df, 'like_best')
dislike_df = explode_reviews(dislike_df, 'dislike')
problems_solving_df = explode_reviews(problems_solving_df, 'problems_solving')

problems_solving_df

,ID,problems_solving
0,8432500,The core problem businesses face is in spreads...
1,8432500,By pushing and automating calculations into a ...
2,8432500,User access is managed at the granular level s...
3,8334645,Anaplan helps our clients connect their Financ...
4,8334645,We hear over and over again from our clients t...
...,...,...
619,123518,Especially working on a sales team like myself...
620,155129,"what ifs in modeling, but having the flexibili..."
621,128309,Creating and consolidating planning financial ...
622,128309,Gained visibility into more the performance of...


Let's add some emeddings for those review questions

In [9]:
!pip install -U sentence-transformers


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [11]:
# Embedding Support
from langchain.embeddings import OpenAIEmbeddings
from sentence_transformers import SentenceTransformer

def generate_embeddings_openai(txt_series):
    embedder = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
    na_filled = txt_series.fillna("", inplace=False) 
    # Generate embeddings for the text column
    return embedder.embed_documents(na_filled.tolist())

def generate_embeddings_free(txt_series):
    embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    na_filled = txt_series.fillna("", inplace=False) 
    # Generate embeddings for the text column
    return embedder.encode(na_filled.tolist())
    

def embed_reviews(df):
    # Your previous function
    # ...

    # Generate embeddings for each column
    for column in ['like_best', 'dislike', 'problems_solving']:
        # Generate embeddings for each column
        if column in df:
            df[f'{column}_embeddings'] = pd.Series(list(generate_embeddings_free(df[column])))
   
    return df

/Users/bornaalmasi/Projects/reviews/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
like_best_embedded =  embed_reviews(like_best_df)
dislike_embedded =  embed_reviews(dislike_df)
problems_solving_embedded =  embed_reviews(problems_solving_df)

now lets cluster the columns using dbscan

In [13]:
!pip install umap-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [14]:
from sklearn.cluster import KMeans
import hdbscan
import numpy as np
import umap

def cluster_column_embeddings(column_embeddings):
    # Initialize HDBSCAN with minimum cluster size
    clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=3, cluster_selection_epsilon=0.5) #gen_min_span_tree=True)
    
    # Fit the HDBSCAN clusterer
    clusterer.fit(column_embeddings)
    
    return clusterer.labels_

def cluster_column_embeddings_umap(column_embeddings):
    # Reduce dimensions with UMAP
    umap_model = umap.UMAP(n_components=10, random_state=42)
    umap_embeddings = umap_model.fit_transform(column_embeddings)
    
    # Initialize HDBSCAN with adjusted parameters
    clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=3)
    
    # Fit the HDBSCAN clusterer on the UMAP-reduced data
    clusterer.fit(umap_embeddings)
    
    return clusterer.labels_

def cluster_column_embeddings_kmeans(column_embeddings, name=''):
    # Using the elbow method to find the optimal number of clusters
    wcss = []  # Within-cluster sum of squares
    max_clusters = 10  # You might want to set this to a different maximum number of clusters

    # Calculating the within-cluster sum of squares for different numbers of clusters
    for i in range(1, max_clusters+1):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
        kmeans.fit(column_embeddings)
        wcss.append(kmeans.inertia_)  # inertia_ is the within-cluster sum-of-squares

    # Plotting the results to visualize the 'elbow'
    plt.figure(figsize=(10,5))
    plt.plot(range(1, max_clusters+1), wcss, marker='o', linestyle='--')
    plt.title(f'{name} Elbow Method For Optimal k')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()

    # You need to choose the optimal number of clusters manually by looking at the plot
    # Let's say you chose the number of clusters to be 'n', you'd then proceed with:
    optimal_n = 4  # Replace with the number of clusters you've determined to be optimal
    kmeans = KMeans(n_clusters=optimal_n, init='k-means++', max_iter=300, n_init=10, random_state=42)
    kmeans.fit(column_embeddings)

    return kmeans.labels_


def cluster_and_append(df_original):
    df = df_original.copy()

    for column in ['like_best_embeddings', 'dislike_embeddings', 'problems_solving_embeddings']:
        if column not in df:
            continue

        # Convert lists of embeddings to a 2D NumPy array
        embeddings_array = np.array(df[column].tolist())
        
        # Ensure embeddings_array is 2D
        if len(embeddings_array.shape) == 1:
            # If the array is 1D, it means each embedding is a single number
            # Reshape it to be 2D with one column
            embeddings_array = embeddings_array.reshape(-1, 1)
        
        # Get cluster labels
        cluster_labels = cluster_column_embeddings_umap(embeddings_array)
        
        # Create new column name for cluster labels
        new_column_name = f'{column}_cluster_id'
        
        # Assign cluster labels to new column in DataFrame
        df[new_column_name] = cluster_labels
    
    return df  # Optional: Return the modified DataFrame

clustered_like_df = cluster_and_append(like_best_embedded)
clustered_dislike_df = cluster_and_append(dislike_embedded)
clustered_problem_df = cluster_and_append(problems_solving_embedded)

/Users/bornaalmasi/Projects/reviews/venv/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/bornaalmasi/Projects/reviews/venv/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
/Users/bornaalmasi/Projects/reviews/venv/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [19]:
import numpy as np
import pandas as pd
import plotly.express as px


def visualize_embeddings(df, embeddings_column, cluster_column, text_column):
    # Extract embeddings, cluster labels, and hover text from DataFrame
    embeddings = np.array(df[embeddings_column].tolist())
    cluster_labels = df[cluster_column]
    review_text = df[text_column]

    # Apply UMAP
    reducer = umap.UMAP(random_state=42)
    embeddings_2d = reducer.fit_transform(embeddings)

    # Create a new DataFrame for the 2D embeddings
    df_2d = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
    df_2d[cluster_column] = cluster_labels.astype(str)  # Ensure that the cluster labels are treated as categorical data
    df_2d['review_text'] = review_text

    # Create the interactive plot
    fig = px.scatter(df_2d, x='x', y='y', color=cluster_column,
                     hover_data={
                         'x': False,  # hide the x-coordinate
                         'y': False,  # hide the y-coordinate
                         cluster_column: False,  # hide the cluster_column
                         'review_text': True  # display the hover_text
                     },
                     title=f'UMAP plot of {embeddings_column} colored by {cluster_column}',
                     labels={'color': 'Cluster ID'})

    # Customize the layout
    fig.update_traces(marker=dict(size=5,
                                  line=dict(width=2,
                                            color='DarkSlateGrey')),
                      selector=dict(mode='markers+text'))

    # Show the plot
    fig.show()

# Usage:
# You would call visualize_embeddings three times, once for each embeddings column.
visualize_embeddings(clustered_like_df, 'like_best_embeddings', 'like_best_embeddings_cluster_id', 'like_best')
visualize_embeddings(clustered_dislike_df, 'dislike_embeddings', 'dislike_embeddings_cluster_id', 'dislike')
visualize_embeddings(clustered_problem_df, 'problems_solving_embeddings', 'problems_solving_embeddings_cluster_id', 'problems_solving')



In [20]:
# Now print the 3 most representative (closest to centroid of each cluster) reviews and print them out for inspection

import numpy as np

def find_closest_to_centroid(df, top_N, embeddings_column, cluster_column, text_column):
    closest_data = {}
    unique_clusters = df[cluster_column].unique()
    for cluster_id in unique_clusters:
        # Filter the data for the current cluster
        cluster_data = df[df[cluster_column] == cluster_id]
        cluster_embeddings = np.array(cluster_data[embeddings_column].tolist())
        
        # Compute the centroid of the current cluster
        centroid = np.mean(cluster_embeddings, axis=0)
        
        # Compute the distances from the centroid to each embedding in the cluster
        distances = np.linalg.norm(cluster_embeddings - centroid, axis=1)
        
        # Get the indices of the N closest embeddings to the centroid
        closest_indices = np.argsort(distances)[:top_N]
        
        # Extract the corresponding text from the DataFrame
        closest_texts = cluster_data.iloc[closest_indices][text_column].tolist()
        
        # Store the closest data
        closest_data[cluster_id] = {
            'indices': closest_indices,
            'texts': closest_texts
        }
    
    return closest_data

def print_report(closest_data):
    for cluster_id, data in closest_data.items():
        print(f'Cluster {cluster_id}:')
        for i, text in enumerate(data['texts'], 1):
            print(f'  Closest {i}: {text}')

# Usage:
# N = 3  # You can replace 3 with any number
# closest_data = find_closest_to_centroid(df, 'embeddings_column', 'cluster_column', 'text_column', N)
# print_report(closest_data)


# Now to print the report

N = 30
print("========= LIKES =========\n\n")
top_likes = find_closest_to_centroid(clustered_like_df, N, 'like_best_embeddings', 'like_best_embeddings_cluster_id', 'like_best')
print_report(top_likes)


print("========= DISLIKES =========\n\n")
top_dislikes = find_closest_to_centroid(clustered_dislike_df, N, 'dislike_embeddings', 'dislike_embeddings_cluster_id', 'dislike')
print_report(top_dislikes)

print("========= USE-CASE =========\n\n")
top_problem_solving = find_closest_to_centroid(clustered_problem_df, N, 'problems_solving_embeddings', 'problems_solving_embeddings_cluster_id', 'problems_solving')
print_report(top_problem_solving)




========= LIKES =========


Cluster 1:
  Closest 1: Anaplan is flexible, powerful and user-friendly.
  Closest 2: Anaplan is a very good tool that helps to design/build models with great flexibility to solve business problems.
  Closest 3: This allows Anaplan to offer agility, flexibility and power to model any Business planning process or workflow across any function.
  Closest 4: Anaplan is really flexible and adaptable to any organization!
  Closest 5: For me, the most helpful aspect of Anaplan is its ability to summarize plan updates real-time.
  Closest 6: Anaplan is an extremely powerful tool that helps organizations break down once siloed/fragmented business functions, streamlining their planning processes and leading to real-time insights that help drive business goals.
  Closest 7: Anaplan is a highly flexible modeling solution, which allows any number of use cases to deploy on it.
  Closest 8: Anaplan is a great connected planning platform and extremely versatile.
  Closest 9

In [21]:
# See how BERTopic does

!pip install bertopic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [23]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")
like_best_topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", vectorizer_model=vectorizer_model,language="english", min_topic_size=10)
topics, probs = like_best_topic_model.fit_transform(like_best_df['like_best'].astype(str))



In [24]:
like_best_topic_model.visualize_barchart()

In [25]:
dislike_topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", vectorizer_model=vectorizer_model,language="english", min_topic_size=10)
topics, probs = dislike_topic_model.fit_transform(dislike_df['dislike'].astype(str))

dislike_topic_model.visualize_documents(dislike_df['dislike'].astype(str))

In [26]:
problem_solving_topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", vectorizer_model=vectorizer_model,language="english", min_topic_size=10)
topics, probs = problem_solving_topic_model.fit_transform(problems_solving_df['problems_solving'].astype(str))

problem_solving_topic_model.visualize_documents(problems_solving_df['problems_solving'].astype(str))

In [27]:
problem_solving_topic_model.visualize_documents(problems_solving_df['problems_solving'].astype(str))

Now let's see if we can take the closest N reviews and use them to identify the topic of the cluster

In [28]:
# Model
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.schema.document import Document
from langchain.chains.summarize import load_summarize_chain
import textwrap
import asyncio

llm3 = ChatOpenAI(
        openai_api_key=OPENAI_KEY,
        temperature=0,
        max_tokens=1000,
        model='gpt-3.5-turbo-16k'
    )
llm4 = ChatOpenAI(
        openai_api_key=OPENAI_KEY,
        temperature=0,
        max_tokens=1000,
        model='gpt-4'
    )
async def summarize_cluster(texts, llm):
    # Use a cheaper model for the map part

    summarize_one_prompt = textwrap.dedent(
        '''
        You are an expert summarizer with the ability to summarize a set of documents into a single concise label.
        Provide at most 3 labels that encapsulate the topics all documents have in common. The documents are enclosed in triple backticks (```).
        The label(s) you provide should not be longer than a few words. Do not include anything else.

        DOCUMENTS:
        ```{review_text}```

        LABEL:
        ''')
    summarize_one_prompt_template = PromptTemplate(template=summarize_one_prompt, input_variables=["review_text"])
    summarize_one_chain = LLMChain(
        llm=llm,
        prompt=summarize_one_prompt_template
    )

    stuffed_reviews_txt = '\n'.join([f'Review {i}: {txt}' for i, txt in enumerate(texts)])
    
    # Assuming summarize_one_chain.run() is an async function
    summary = await summarize_one_chain.arun(stuffed_reviews_txt)
    return summary

async def summarize_parallel(top_n_cluster):
    # Create a mapping from coroutine to cluster_id
    coro_to_cluster_id = {summarize_cluster(vals['texts'], llm3): cluster_id for cluster_id, vals in top_n_cluster.items()}
    coroutines = list(coro_to_cluster_id.keys())

    # Gather results from all coroutines
    results = await asyncio.gather(*coroutines)

    # Pair each result with its corresponding cluster ID
    for coro, result in zip(coroutines, results):
        top_n_cluster[coro_to_cluster_id[coro]]['cluster_label'] = result
        top_n_cluster[-1]['cluster_label'] = 'Uncategorized'

    return top_n_cluster


await asyncio.gather(
    summarize_parallel(top_likes)
    ,summarize_parallel(top_dislikes),
    summarize_parallel(top_problem_solving)
)

Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..
Retrying langchain.chat_models.openai.acompletion_

[{1: {'indices': array([129,  71, 170,  13, 142,  60, 143,  34, 195,  11,  47,  84, 155,
          177, 154,  81,  46,  41,  80,  29,  78,  87, 137,  19,  76,  57,
          134,  39, 163, 148]),
   'texts': ['Anaplan is flexible, powerful and user-friendly.',
    'Anaplan is a very good tool that helps to design/build models with great flexibility to solve business problems.',
    'This allows Anaplan to offer agility, flexibility and power to model any Business planning process or workflow across any function.',
    'Anaplan is really flexible and adaptable to any organization!',
    'For me, the most helpful aspect of Anaplan is its ability to summarize plan updates real-time.',
    'Anaplan is an extremely powerful tool that helps organizations break down once siloed/fragmented business functions, streamlining their planning processes and leading to real-time insights that help drive business goals.',
    'Anaplan is a highly flexible modeling solution, which allows any number of u

In [31]:

top_likes_map = {cluster_id: data["cluster_label"] for cluster_id, data in top_likes.items()}
top_dislikes_map = {cluster_id: data["cluster_label"] for cluster_id, data in top_dislikes.items()}
top_problem_solving_map = {cluster_id: data["cluster_label"] for cluster_id, data in top_problem_solving.items()}

# Create a new column 'cluster_label' by mapping 'cluster_id' to 'cluster_label'
clustered_like_df['like_cluster_label'] = clustered_like_df['like_best_embeddings_cluster_id'].map(top_likes_map)
clustered_dislike_df['dislike_cluster_label'] = clustered_dislike_df['dislike_embeddings_cluster_id'].map(top_dislikes_map)
clustered_problem_df['problem_solving_label'] = clustered_problem_df['problems_solving_embeddings_cluster_id'].map(top_problem_solving_map)

In [32]:
visualize_embeddings(clustered_like_df, 'like_best_embeddings', 'like_cluster_label', 'like_best')
visualize_embeddings(clustered_dislike_df, 'dislike_embeddings', 'dislike_cluster_label', 'dislike')
visualize_embeddings(clustered_problem_df, 'problems_solving_embeddings', 'problem_solving_label', 'problems_solving')